## Neural Machine Translation

## Introduction
In this notebook, we will build a deep neural network that functions as part of an end-to-end machine translation pipeline. The completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - converts text to sequence of integers.
- **Models** models which accepts a sequence of integers as input and returns a probability distribution over possible translations.
- **Prediction** Run the model prediction on English text.

In [1]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [3]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

### Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a Udacity Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".
- If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
- If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13799542385446444587
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 357302272
locality {
  bus_id: 1
}
incarnation: 7820372635673363647
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0"
]


## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary to be able to train model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [5]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [6]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [7]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, we convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

In [8]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    tokenized_sentences = tokenizer.texts_to_sequences(x)

    return tokenized_sentences, tokenizer
tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [9]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    padded_sequences = pad_sequences(x, maxlen=length, padding='post')
    return padded_sequences
tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline


In [16]:
preproc_english_sentences

array([[17, 23,  1, ..., 44,  0,  0],
       [ 5, 20, 21, ..., 51,  2, 45],
       [22,  1,  9, ..., 34,  0,  0],
       ..., 
       [24,  1, 10, ..., 54,  0,  0],
       [ 5, 84,  1, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0]], dtype=int32)

In [19]:
preprocess_x, x_tk = tokenize(english_sentences)
preprocess_x = pad(preprocess_x, length=21)

In [20]:
preprocess_x

array([[17, 23,  1, ...,  0,  0,  0],
       [ 5, 20, 21, ...,  0,  0,  0],
       [22,  1,  9, ...,  0,  0,  0],
       ..., 
       [24,  1, 10, ...,  0,  0,  0],
       [ 5, 84,  1, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0]], dtype=int32)

In [10]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


## Models
In this section, we will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, we will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  We'll be using this function to better understand the output of the neural network.

In [11]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, we'll build a RNN that translates English to French.

In [12]:
from keras.models import Sequential
from keras.layers import Dense, Activation, GRU, RepeatVector, TimeDistributed, LSTM

def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    HIDDEN_SIZE = 64
    
    model = Sequential()
    model.add(LSTM(HIDDEN_SIZE, input_shape=input_shape[1:], return_sequences=True))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))

    lr = 1e-3
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(lr),
                  metrics=['accuracy'])
    return model
tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 9s 82us/step - loss: 3.4119 - acc: 0.4158 - val_loss: nan - val_acc: 0.4615
Epoch 2/10
110288/110288 [==============================] - 8s 74us/step - loss: 2.4222 - acc: 0.4700 - val_loss: nan - val_acc: 0.4915
Epoch 3/10
110288/110288 [==============================] - 8s 75us/step - loss: 2.1365 - acc: 0.5098 - val_loss: nan - val_acc: 0.5341
Epoch 4/10
110288/110288 [==============================] - 8s 75us/step - loss: 1.9183 - acc: 0.5485 - val_loss: nan - val_acc: 0.5617
Epoch 5/10
110288/110288 [==============================] - 8s 75us/step - loss: 1.7799 - acc: 0.5716 - val_loss: nan - val_acc: 0.5795
Epoch 6/10
110288/110288 [==============================] - 8s 74us/step - loss: 1.6870 - acc: 0.5846 - val_loss: nan - val_acc: 0.5933
Epoch 7/10
110288/110288 [==============================] - 8s 75us/step - loss: 1.6185 - acc: 0.5963 - val_loss: nan - val_acc: 0.59

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
We've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, we'll create a RNN model using embedding.

In [13]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    EMBED_SIZE = 128
    HIDDEN_SIZE = 64

    model = Sequential()
    model.add(Embedding(input_dim=english_vocab_size, output_dim=EMBED_SIZE, input_length=input_shape[1]))
    model.add(LSTM(HIDDEN_SIZE, return_sequences=True))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))

    lr = 1e-3
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(lr),
                  metrics=['accuracy'])
    return model
tests.test_embed_model(embed_model)

# Reshaping the input to work with a embedding rnn
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape(-1, preproc_french_sentences.shape[1])

# Train the neural network
embed_model = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
embed_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(embed_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 9s 85us/step - loss: 3.6743 - acc: 0.4013 - val_loss: nan - val_acc: 0.4095
Epoch 2/10
110288/110288 [==============================] - 9s 81us/step - loss: 2.5690 - acc: 0.4594 - val_loss: nan - val_acc: 0.5000
Epoch 3/10
110288/110288 [==============================] - 9s 80us/step - loss: 2.0127 - acc: 0.5325 - val_loss: nan - val_acc: 0.5801
Epoch 4/10
110288/110288 [==============================] - 9s 81us/step - loss: 1.6111 - acc: 0.6117 - val_loss: nan - val_acc: 0.6459
Epoch 5/10
110288/110288 [==============================] - 9s 81us/step - loss: 1.3175 - acc: 0.6745 - val_loss: nan - val_acc: 0.7040
Epoch 6/10
110288/110288 [==============================] - 9s 81us/step - loss: 1.0788 - acc: 0.7318 - val_loss: nan - val_acc: 0.7550
Epoch 7/10
110288/110288 [==============================] - 9s 80us/step - loss: 0.9076 - acc: 0.7673 - val_loss: nan - val_acc: 0.78

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [14]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    HIDDEN_SIZE = 64

    model = Sequential()
    model.add(Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True), input_shape=(input_shape[1:])))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))

    lr = 1e-3
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(lr),
                  metrics=['accuracy'])
    return model
tests.test_bd_model(bd_model)

# TODO: Train and Print prediction(s)

# Reshaping the input to work with a bidirectional rnn
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
bd_model = bd_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
bd_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(bd_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 13s 116us/step - loss: 3.2103 - acc: 0.4414 - val_loss: nan - val_acc: 0.5003
Epoch 2/10
110288/110288 [==============================] - 12s 110us/step - loss: 1.9604 - acc: 0.5350 - val_loss: nan - val_acc: 0.5662
Epoch 3/10
110288/110288 [==============================] - 12s 110us/step - loss: 1.6673 - acc: 0.5789 - val_loss: nan - val_acc: 0.5918
Epoch 4/10
110288/110288 [==============================] - 12s 109us/step - loss: 1.5336 - acc: 0.6043 - val_loss: nan - val_acc: 0.6109
Epoch 5/10
110288/110288 [==============================] - 12s 110us/step - loss: 1.4508 - acc: 0.6143 - val_loss: nan - val_acc: 0.6178
Epoch 6/10
110288/110288 [==============================] - 12s 110us/step - loss: 1.3896 - acc: 0.6229 - val_loss: nan - val_acc: 0.6278
Epoch 7/10
110288/110288 [==============================] - 12s 109us/step - loss: 1.3407 - acc: 0.6309 - val_loss: nan -

### Model 4: Encoder-Decoder
This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

In [ ]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # OPTIONAL: Implement
    HIDDEN_SIZE=64
    
    model = Sequential()
    model.add(LSTM(HIDDEN_SIZE, input_shape=(output_sequence_length, 1)))
    model.add(RepeatVector(input_shape[1]))
    model.add(LSTM(HIDDEN_SIZE, return_sequences=True))
    model.add(TimeDistributed(Dense(french_vocab_size)))

    lr = 1e-3
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(lr),
                  metrics=['accuracy'])
    return model
tests.test_encdec_model(encdec_model)

# OPTIONAL: Train and Print prediction(s)


### Model 5: Custom
Finally we'll create a model that incorporates embedding and a bidirectional rnn into one model.

In [36]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    EMBED_SIZE = 128
    HIDDEN_SIZE = 64

    model = Sequential()
    model.add(Embedding(input_dim=english_vocab_size, output_dim=EMBED_SIZE, input_length=input_shape[1]))
    model.add(Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True, recurrent_dropout=0.3)))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))

    lr = 1e-3
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(lr),
                  metrics=['accuracy'])
    return model
tests.test_model_final(model_final)

print('Final Model Loaded')

# TODO: Train the final model

# Reshaping the input to work with a final model
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape(-1, preproc_french_sentences.shape[1])

# Train the neural network
final_model = model_final(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
final_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(final_model.predict(tmp_x[:1])[0], french_tokenizer))

Final Model Loaded
Train on 110288 samples, validate on 27573 samples
Epoch 1/20
110288/110288 [==============================] - 19s 174us/step - loss: 3.3506 - acc: 0.4307 - val_loss: nan - val_acc: 0.4843
Epoch 2/20
110288/110288 [==============================] - 17s 153us/step - loss: 2.1923 - acc: 0.5083 - val_loss: nan - val_acc: 0.5533
Epoch 3/20
110288/110288 [==============================] - 17s 152us/step - loss: 1.6312 - acc: 0.6001 - val_loss: nan - val_acc: 0.6573
Epoch 4/20
110288/110288 [==============================] - 17s 152us/step - loss: 1.1920 - acc: 0.6948 - val_loss: nan - val_acc: 0.7357
Epoch 5/20
110288/110288 [==============================] - 17s 152us/step - loss: 0.8966 - acc: 0.7608 - val_loss: nan - val_acc: 0.7839
Epoch 6/20
110288/110288 [==============================] - 17s 152us/step - loss: 0.7335 - acc: 0.7950 - val_loss: nan - val_acc: 0.8072
Epoch 7/20
110288/110288 [==============================] - 17s 155us/step - loss: 0.6331 - acc: 0.816

## Prediction

In [35]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    max_english_sequence_length = x.shape[1]
    max_french_sequence_length = y.shape[1]
    english_vocab_size = len(x_tk.word_index)
    french_vocab_size = len(y_tk.word_index)
    
    x = pad(x, max_french_sequence_length)
    x = x.reshape((-1, y.shape[-2]))

    # Train the neural network
    model = model_final(
        tmp_x.shape,
        max_french_sequence_length,
        english_vocab_size,
        french_vocab_size)
    model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=40, validation_split=0.2)
        
    x_id_to_word = {value: key for key, value in x_tk.word_index.items()}
    x_id_to_word[0] = '<PAD>'

    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    
    #print(' '.join([x_id_to_word[np.max(x)] for x in sentences[1]]))
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))

final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Train on 110288 samples, validate on 27573 samples
Epoch 1/40
110288/110288 [==============================] - 19s 169us/step - loss: 3.3074 - acc: 0.4414 - val_loss: nan - val_acc: 0.4797
Epoch 2/40
110288/110288 [==============================] - 17s 151us/step - loss: 2.1732 - acc: 0.5052 - val_loss: nan - val_acc: 0.5359
Epoch 3/40
110288/110288 [==============================] - 17s 151us/step - loss: 1.6835 - acc: 0.5895 - val_loss: nan - val_acc: 0.6288
Epoch 4/40
110288/110288 [==============================] - 17s 151us/step - loss: 1.2898 - acc: 0.6698 - val_loss: nan - val_acc: 0.7055
Epoch 5/40
110288/110288 [==============================] - 17s 152us/step - loss: 1.0016 - acc: 0.7374 - val_loss: nan - val_acc: 0.7618
Epoch 6/40
110288/110288 [==============================] - 17s 151us/step - loss: 0.8174 - acc: 0.7771 - val_loss: nan - val_acc: 0.7921
Epoch 7/40
110288/110288 [==============================] - 17s 153us/step - loss: 0.6942 - acc: 0.8042 - val_loss: nan -